# Agregaciones

In [8]:
import pymongo
import re
client = pymongo.MongoClient(f"mongodb://{user}:{password}@{proyecto}-shard-00-00.mmsfi.mongodb.net:27017,{proyecto}-shard-00-01.mmsfi.mongodb.net:27017,{proyecto}-shard-00-02.mmsfi.mongodb.net:27017/test?ssl=true&replicaSet=atlas-11i31k-shard-0&authSource=admin&retryWrites=true&w=majority")
db = client.test

### Reto 1: Agrupamientos
Con base en el ejemplo 1, modifica el agrupamiento para que muestre el costo promedio por habitación por país de las propiedades de tipo casa.

In [9]:
result = client['sample_airbnb']['listingsAndReviews'].aggregate([
    {
        '$match': {
            'property_type': 'House', 
            'bedrooms': {
                '$gte': 1
            }
        }
    }, {
        '$addFields': {
            'costoRecamara': {
                '$divide': [
                    '$price', '$bedrooms'
                ]
            }
        }
    }, {
        '$group': {
            '_id': '$address.country', 
            'recamaras': {
                '$sum': 1
            }, 
            'total': {
                '$sum': '$costoRecamara'
            }
        }
    }, {
        '$addFields': {
            'pais': '$_id', 
            'costoPromedio': {
                '$trunc': [
                    {
                        '$divide': [
                            '$total', '$recamaras'
                        ]
                    }, 2
                ]
            }
        }
    }, {
        '$project': {
            '_id': 0, 
            'pais': 1, 
            'costoPromedio': 1
        }
    }
])

In [10]:
i= 0
for x in result:
    print('pais: ', x['pais'])
    print('costo Promedio: ', x['costoPromedio'])
    print()
    i+=1
    if i > 5:
        break

pais:  China
costo Promedio:  393.00

pais:  United States
costo Promedio:  113.73

pais:  Brazil
costo Promedio:  348.66

pais:  Turkey
costo Promedio:  209.87

pais:  Hong Kong
costo Promedio:  514.60

pais:  Canada
costo Promedio:  114.98



### Reto 2: Asociación de colecciones
Usando las colecciones comments y users, se requiere conocer el correo y contraseña de cada persona que realizó un comentario. Construye un pipeline que genere como resultado estos datos.

In [11]:
result = client['sample_mflix']['comments'].aggregate([
    {
        '$lookup': {
            'from': 'users', 
            'localField': 'email', 
            'foreignField': 'email', 
            'as': 'usuario'
        }
    }, {
        '$addFields': {
            'usuario': {
                '$arrayElemAt': [
                    '$usuario', 0
                ]
            }
        }
    }, {
        '$addFields': {
            'password': '$usuario.password'
        }
    }, {
        '$project': {
            '_id': 0, 
            'text': 1, 
            'email': 1, 
            'password': 1
        }
    }
])

In [12]:
i= 0
for x in result:
    print('text: ', x['text'])
    print('email: ', x['email'])
    print('password: ', x['password'])
    print()
    i+=1
    if i > 5:
        break

text:  Minus sequi incidunt cum magnam. Quam voluptatum vitae ab voluptatum cum. Autem perferendis nisi nulla dolores aut recusandae.
email:  iain_glen@gameofthron.es
password:  $2b$12$K8bKkwnpkrjsBPzASZxO/.yj7d9kvupiVtO6JA3Xl106AKXr3pXFK

text:  Ab consequatur numquam sed eligendi ex unde. Dolorem illum minima numquam dicta ipsa magnam nostrum. Possimus sed inventore cum non.
email:  sarah_lewis@fakegmail.com
password:  $2b$12$5kCUjcP3lvYSzhouVJTpOeCZ7e7Xke8gDoPPg2Uyz39tNKrp9om1a

text:  Harum necessitatibus rerum ab voluptatem nulla at reiciendis recusandae. Quas ullam similique atque sit. A voluptates labore nam illum dolorem qui.
email:  kristian_nairn@gameofthron.es
password:  $2b$12$hFkSycBpMDENs/sT.5y0feifE5IluHUJmYHqH/lSkcc81/HRzzRSq

text:  Doloremque corporis eaque illum nihil. Veritatis tenetur quo vitae ipsa nihil facere dolorum esse. Minima inventore ullam laudantium vitae a nesciunt. Architecto laborum eveniet quam.
email:  brenock_o'connor@gameofthron.es
password:  $2b$1

### Reto 3: Generación de vistas
Usando el pipeline que generaste en el Reto 2, genera la vista correspondiente.